## Assignment 8

In [1]:
%matplotlib inline

import numpy as np

import pystan
import stan_utility

import matplotlib
import matplotlib.pyplot as plt

font = {'size': 16}

matplotlib.rc('font', **font)

print('numpy', np.__version__)
print('pystan', pystan.__version__)

numpy 1.14.1
pystan 2.17.1.0


### Decision analysis for the factory data

Some configuration values and utility functions:

In [5]:
product_cost       = 100
product_price      = 200
quality_threshold  = 85

In [7]:
def utility(x):    
    if x < quality_threshold:
        return -product_cost
    else:
        return product_price - product_cost

We load the data

In [3]:
data_factory = np.loadtxt('../data/factory.txt')
data_factory.shape

(5, 6)

#### Model
We will use the hierarchical model since it performed the best in the previous assignments.

In [8]:
stan_code_hierarchical = """
data {
  int<lower=0> N;            // number of data points
  int<lower=0> K;            // number of groups
  int<lower=1,upper=K> x[N]; // group indicator
  vector[N] y;               // target
}
parameters {
  real mu_prior;             // shared prior mean
  real<lower=0> sigma_prior; // shared prior std
  vector[K] mu;              // group means
  real<lower=0> sigma;       // shared std
}
model {
  for (k in 1:K) {
    mu[k] ~ normal(mu_prior, sigma_prior);
  }
  y ~ normal(mu[x], sigma);
}
generated quantities {
}
"""

model_hierarchical = pystan.StanModel(model_code = stan_code_hierarchical)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c021d9288ff6eb39578ac2232b0d58fb NOW.


In [9]:
x = np.tile(np.arange(1, data_factory.shape[1] + 1), data_factory.shape[0])
y = data_factory.flatten()
N = len(x)
K = np.max(x)

fit_hierarchical = model_hierarchical.sampling(
    data = {
        'N': N,
        'K': K,
        'x': x,
        'y': y
    })

c:\users\ncp\appdata\local\continuum\anaconda3\envs\stan_env\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


We check the fitted parameters and the performance of the fit, i.e. the treedepth, E-BFMI, and divergences:

In [66]:
stan_utility.check_treedepth(fit_hierarchical)
stan_utility.check_energy(fit_hierarchical)
stan_utility.check_div(fit_hierarchical)

0 of 4000 iterations saturated the maximum tree depth of 10 (0.0%)
21.0 of 4000 iterations ended with a divergence (0.525%)
Try running with larger adapt_delta to remove the divergences


We see some iterations ended with a divergence, but since it is such small fraction, we accept the performance.

In [12]:
params = fit_hierarchical.extract()

In [19]:
vec_utility = np.vectorize(utility)


In [22]:
vec_utility(params['mu']).mean(axis = 0)

array([-54.55,  98.75,  48.1 ,  99.95,  64.35,  34.8 ])

#### Conclusions